# Kaggle

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'wikibooks-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1167113%2F2730445%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240723%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240723T144735Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D33216a71149278a6ac81347f67418e985cc50958ce8bd5ed19bde5170ba0830fd9683e3c09295706165707c8addb24eff813d0d2cf753532e27156a32a0b728295fd16bebfe7b1f3658cf4cd03a43688fce5cae05ad2b2960ab22e17f4d343f77c06276b1b70151267453b5b5cd2bf5908f55a70ae77e57da598197d399a7f0405cbc7b2fbefa6156d974e825d32c730b1aa65b592e6e64320d568696d0f4aafb46d29808b931443a0664544dfc24fa1276d5ac38857b3957b7846a8798d0585bb1af46e81163fa2648b2edd22d34cef8d5ed999d7f9bb154efba538634174edb9442b482ff11cff1699bd696a69c7463305de6311eb3e4c1c0e87c0de6dac26,daigt-proper-train-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3942644%2F6890527%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240723%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240723T144735Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D25b4d396beaccacff4ce8794b33f119f9f877fd8d99da1a766e3e60649bfab421a742b0843d83f1858b75cf6863a4869e0b17da1f30e8e63c49cc4f2c3986c6327624e76eda7f0d4e86162e751f8e8cacd89b5dd9cd2c9c1b4c2c0822abdc4072979fd045a794ba8e9dd2013ffa19bd8760474c52aa477a7f2f5eef38d724d6f313d37ef911bca1945b58cb756b8e8b32dd02153b7cc89bf0ae47754c1144f78fffb9ed2960febef06d503e92db917048c266c9749f4551f8ecc4b360a63081a693b2de3d1970142c3348f66b8a8a660f4e8fe8307365744b8fad1401b5f5fa4e9ac6f7da0c36b08abc0e77ac16d1af8ff007cb4bf9d59080be46aecd978809f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/daigt-proper-train-dataset/train_drcat_03.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv


---

# Packages

In [3]:
# !pip install spacy
# !python -m spacy download en_core_web_trf
# !python -m spacy download en_core_web_lg

In [4]:
!pip install keras_preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import tensorflow as tf

from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Flatten, Dense,Dropout,LSTM,Embedding

from tensorflow.keras.callbacks import EarlyStopping


from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.text import Tokenizer


from keras_preprocessing.sequence import pad_sequences


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from IPython.display import clear_output

2024-07-30 16:12:49.169134: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 16:12:49.169300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 16:12:49.401445: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import  word_tokenize,sent_tokenize,PunktSentenceTokenizer
from nltk.stem.porter import *
from nltk.corpus import state_union
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer,SnowballStemmer
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import CountVectorizer

import spacy
from spacy import displacy

from scipy import spatial

In [7]:
def progress(cnt,y,refresh=300):
    if not cnt % refresh:
        clear_output(wait=True)
        progress = cnt * 100 / y

        print("\033[92m━\033[0m"*int(progress/1.7),end="")
        print("-"*int((100 - progress)/1.7),end="")
        print(f" \033[92m{progress:.2f}%\033[0m")

# ***Text Generation***
---

# I- Data


In [8]:
data_01 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv')
data_02 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv')
data_03 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_03.csv')
data_04 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv')
data_01.head()

,text,label,source,fold
0,There are alot reasons to keep our the despise...,0,persuade_corpus,2
1,Driving smart cars that drive by themself has ...,0,persuade_corpus,4
2,"Dear Principal,\n\nI believe that students at ...",0,persuade_corpus,0
3,"Dear Principal,\n\nCommunity service should no...",0,persuade_corpus,0
4,My argument for the development of the driverl...,0,persuade_corpus,3


In [9]:
data_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33259 entries, 0 to 33258
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    33259 non-null  object
 1   label   33259 non-null  int64 
 2   source  33259 non-null  object
 3   fold    33259 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.0+ MB


In [10]:
data_01.dropna()

,text,label,source,fold
0,There are alot reasons to keep our the despise...,0,persuade_corpus,2
1,Driving smart cars that drive by themself has ...,0,persuade_corpus,4
2,"Dear Principal,\n\nI believe that students at ...",0,persuade_corpus,0
3,"Dear Principal,\n\nCommunity service should no...",0,persuade_corpus,0
4,My argument for the development of the driverl...,0,persuade_corpus,3
...,...,...,...,...
33254,Are driverless cars really necessary? Most of ...,0,persuade_corpus,4
33255,"""Oh man I didn't make the soccer team!"", yelle...",0,persuade_corpus,1
33256,I believe that using this technology could be ...,0,persuade_corpus,0
33257,Texting & Driving\n\nUsing your phone while dr...,0,persuade_corpus,3


In [11]:
data_01.text[100]

'Some schools offer distance schooling for students to be able to learn from home, and would they benefit from it. Should more schools do this where you live. Here are some advantages to schools that do this.\n\nOne advantage to doing distance learning is there is no traveling. Students donÃ\x82Â´t have to come to school everyday, and there parents donÃ\x82Â´t have to bring them either. It is just allot less running around for the students and parents. They dont have to get up so early just to go to school. You just get right out of bed and get your work done.\n\nSchools are doing it now so kids can spend more time with their family. They have to go to school for 8 hours a day ,then come home to spend little time with the family. Kids need that family time rather then going to school all day. The parents can help them do the work if needed. Just getting more support then at school.\n\nI feel like if kids did their work at home they would get more work one. They dont have here friends t

In [12]:
lb_0 = [data_01.text[x] for x in range(len(data_01.text)) if data_01.label[x] == 0]
lb_1 = [data_01.text[x] for x in range(len(data_01.text)) if data_01.label[x] == 1]

In [13]:
max_txt = len((max(lb_0,key=lambda x : len(x.split()))).split())
max_txt

1656

In [14]:
sampel = lb_0[:1000]

> i will use only sample from data , to speed up the prosees


## I- 02 data prosising : 


In [15]:
#del  \xa0
filter_01 = [re.sub("\\[A-z]*?[0-9]+","",s) for s in sampel]

In [16]:
tokenized = [word_tokenize(filter_01[i].lower()) for i in range(len(filter_01))]

> it is not wise to convert textes to lower case , but i am forced due to the small size of dataset


## GloVe

In [17]:
!wget https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip

--2024-07-30 16:13:30--  https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip
Resolving huggingface.co (huggingface.co)... 18.239.50.80, 18.239.50.49, 18.239.50.16, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.80|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/stanfordnlp/glove/6471382cdd837544bf3ac72497a38715e845897d265b2b424b4761832009c837?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27glove.6B.zip%3B+filename%3D%22glove.6B.zip%22%3B&response-content-type=application%2Fzip&Expires=1722615211&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMjYxNTIxMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9zdGFuZm9yZG5scC9nbG92ZS82NDcxMzgyY2RkODM3NTQ0YmYzYWM3MjQ5N2EzODcxNWU4NDU4OTdkMjY1YjJiNDI0YjQ3NjE4MzIwMDljODM3P3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Signature=IXNy%7ECqyzx2lApIcJHvQo6N9OAaxtSi

In [18]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [19]:
!ls -lh

total 2.9G
-rw-rw-r-- 1 root root 332M Dec 22  2015 glove.6B.100d.txt
-rw-rw-r-- 1 root root 662M Dec 22  2015 glove.6B.200d.txt
-rw-rw-r-- 1 root root 990M Dec 22  2015 glove.6B.300d.txt
-rw-rw-r-- 1 root root 164M Dec 22  2015 glove.6B.50d.txt
-rw-r--r-- 1 root root 823M Aug 14  2021 glove.6B.zip


In [20]:
!head glove.6B.50d.txt

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581
, 0.013441 0.23682 -0.16899 0.40951 0.63812 0.47709 -0.42852 -0.55641 -0.364 -0.23938 0.13001 -0.063734 -0.39575 -0.48162 0.23291 0.090201 -0.13324 0.078639 -0.41634 -0.15428 0.10068 0.48891 0.31226 -0.1252 -0.037512 -1.5179 0.12612 -0.02442 -0.042961 -0.28351 3.5416 -0.11956 -0.014533 -0.1499 0.21864 -0.33412 -0.13872 0.31806 0.70358 0.44858 -0.080262 0.63003 0.32111 -0.46765 0.22786 0.36034 -0.37818 -0.56657 0.044691 0.30392
. 0.15164 0.30177 -0.16763 0.17684 0.31719 0.33973 -0.43478 -0.31086 -0.44999 -0.29486 0.16608 0.11963 -0.41328 -0.42353

In [21]:
def load_glove_vectors(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()

            try:
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
            except:
              # red exseption
                print("\033[91m",'='*20,"\033[0m")
                continue
            embeddings_index[word] = coefs
    return embeddings_index

# Specify the path to the GloVe file
glove_file = './glove.6B.50d.txt'

# Load the GloVe vectors
glove = load_glove_vectors(glove_file)

In [22]:
len(glove),glove["hi"].shape

(400001, (50,))

In [23]:
glove_keys = list(glove.keys())

In [24]:
len(glove_keys)

400001

In [25]:
embedding_dim = 50
vocab_size = len(glove) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

word_index = {word: i for i, word in enumerate(glove.keys(), 1)}
for word, i in word_index.items():
    embedding_vector = glove.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [26]:
embedding_matrix.shape

(400002, 50)

## Glove dict


In [27]:
glove_keys[0]

'the'

In [28]:
index2word = {}
word2index = {}


for i in range(len(glove_keys)):
    index2word[i+1] = glove_keys[i]
    word2index[glove_keys[i]] = i+1

## Text-To-Seq

In [29]:
txt_seq = []

cnt = 0
for sents in tokenized:
    progress(cnt,1000,10)
    sent = []
    for word in sents : 
        try:
            sent.append(word2index[word])
        except:
            word2index["<unk>"]
    txt_seq.append(sent)
    cnt += 1

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.00%


## padding 

In [30]:
txt_seq_pad = pad_sequences(txt_seq,padding='post')

## N-Grams : 

In [34]:
ng_data = nltk.ngrams(txt_seq,3)

In [39]:
ng_list = [list(nltk.ngrams(sent,3)) for sent in txt_seq]

In [69]:
X = [list(x[0][:2]) for x in ng_list]
Y = [x[0][2] for x in ng_list]

In [91]:
X = np.array(X)
Y = np.array(Y)

## split data

In [92]:
X_train , X_test ,y_train , y_test  = train_test_split(X,Y,test_size=0.2)

# II.model

In [149]:
model = Sequential ([
    Embedding(input_dim=vocab_size,
                output_dim=embedding_dim,
                weights=[embedding_matrix],
                input_length=2,
                trainable=False),
    LSTM(512,return_sequences=True),
    Dropout(0.2),
    LSTM(256,return_sequences=True),
    Dropout(0.2),
    LSTM(128),
    Dense(vocab_size, activation='softmax')
])

In [150]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [151]:
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)        │ ?                      │    20,000,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_19 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_20 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_21 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,000,100 (76.29 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 20,000,100 (76.29 MB)

In [152]:
X_train.shape,y_train.shape,y_train_categorical.shape

((800, 2), (800,), (800, 400002))

In [153]:
from keras.utils import to_categorical

y_train_categorical = to_categorical(y_train, num_classes=400002)
y_test_categorical = to_categorical(y_test, num_classes=400002)

In [154]:
early_stopping_acc = EarlyStopping(monitor='val_accuracy',min_delta=0.0001, patience=1, restore_best_weights=True)
early_stopping_loss = EarlyStopping(monitor='val_loss',min_delta=0.0001, patience=1, restore_best_weights=True)

history = model.fit(X_train,y_train_categorical,epochs=10 , validation_data=(X_test, y_test_categorical), callbacks=[early_stopping_acc,early_stopping_loss])

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.1052 - loss: 12.8331 - val_accuracy: 0.1450 - val_loss: 11.3974
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 922ms/step - accuracy: 0.1140 - loss: 10.0719 - val_accuracy: 0.1450 - val_loss: 7.4124
